In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L4\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['Capital.csv', 'Central.csv', 'Dunwoody.csv', 'Genese.csv', 'Hudson.csv', 'LongIsland.csv', 'Milwood.csv', 'NYC.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[0]))

,Demand,Normalized_net,Temprature
0,1280.8,0.531006,-5.8
1,1230.8,0.510277,-6.1
2,1198.5,0.496886,-7.5
3,1188.9,0.492906,-8.4
4,1190.9,0.493735,-7.4
...,...,...,...
43819,1452.6,0.602233,3.4
43820,1399.7,0.580301,3.3
43821,1339.0,0.555136,2.6
43822,1280.9,0.531048,2.7


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[0]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temprature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_Capital,Temp_Capital
0,1280.8,-5.8
1,1230.8,-6.1
2,1198.5,-7.5
3,1188.9,-8.4
4,1190.9,-7.4
...,...,...
43819,1452.6,3.4
43820,1399.7,3.3
43821,1339.0,2.6
43822,1280.9,2.7


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_Capital,Temp_Capital,Lag-2,Lag-7
0,1529.9,-13.9,1319.8,1280.8
1,1538.5,-14.2,1291.2,1230.8
2,1478.5,-14.8,1273.3,1198.5
3,1456.9,-15.6,1277.6,1188.9
4,1484.0,-16.0,1313.5,1190.9
...,...,...,...,...
43651,1452.6,3.4,1502.3,1394.4
43652,1399.7,3.3,1453.5,1375.5
43653,1339.0,2.6,1381.0,1351.5
43654,1280.9,2.7,1288.4,1304.1


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 21s 9ms/step - loss: 0.0120 - mape: 5918.5137 - mae: 0.0850 - val_loss: 0.0077 - val_mape: 43.9476 - val_mae: 0.0728
Epoch 2/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0070 - mape: 5536.7080 - mae: 0.0646 - val_loss: 0.0059 - val_mape: 36.7389 - val_mae: 0.0610
Epoch 3/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0050 - mape: 5366.9302 - mae: 0.0544 - val_loss: 0.0062 - val_mape: 36.2428 - val_mae: 0.0614
Epoch 4/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0043 - mape: 4910.0586 - mae: 0.0502 - val_loss: 0.0057 - val_mape: 29.9281 - val_mae: 0.0573
Epoch 5/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0038 - mape: 4880.7163 - mae: 0.0472 - val_loss: 0.0077 - val_mape: 28.0757 - val_mae: 0.0663
Epoch 6/100
1105/1105 [==============================] - 10s 9ms/step - loss: 0.0035 - mape: 4134.0942 - mae: 0.0453 - val_loss: 0.006

1105/1105 [==============================] - 9s 8ms/step - loss: 0.0018 - mape: 2400.1069 - mae: 0.0319 - val_loss: 0.0057 - val_mape: 27.4353 - val_mae: 0.0553
Epoch 49/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0017 - mape: 2191.0129 - mae: 0.0314 - val_loss: 0.0048 - val_mape: 24.6999 - val_mae: 0.0501
Epoch 50/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0017 - mape: 2442.1079 - mae: 0.0311 - val_loss: 0.0049 - val_mape: 27.6434 - val_mae: 0.0517
Epoch 51/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0017 - mape: 2485.5957 - mae: 0.0311 - val_loss: 0.0052 - val_mape: 26.6826 - val_mae: 0.0514
Epoch 52/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0016 - mape: 2240.5159 - mae: 0.0306 - val_loss: 0.0049 - val_mape: 24.9476 - val_mae: 0.0502
Epoch 53/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0016 - mape: 2197.0205 - mae: 0.0305 - val_loss: 0.0052 - val_m

Epoch 95/100
1105/1105 [==============================] - 8s 8ms/step - loss: 9.1377e-04 - mape: 1535.0134 - mae: 0.0231 - val_loss: 0.0058 - val_mape: 29.2765 - val_mae: 0.0573
Epoch 96/100
1105/1105 [==============================] - 8s 8ms/step - loss: 9.0287e-04 - mape: 1595.0557 - mae: 0.0229 - val_loss: 0.0064 - val_mape: 28.1751 - val_mae: 0.0587
Epoch 97/100
1105/1105 [==============================] - 8s 8ms/step - loss: 8.9415e-04 - mape: 1651.1647 - mae: 0.0228 - val_loss: 0.0061 - val_mape: 28.2667 - val_mae: 0.0580
Epoch 98/100
1105/1105 [==============================] - 9s 8ms/step - loss: 8.7867e-04 - mape: 1228.6152 - mae: 0.0226 - val_loss: 0.0063 - val_mape: 27.2681 - val_mae: 0.0578
Epoch 99/100
1105/1105 [==============================] - 9s 8ms/step - loss: 8.6958e-04 - mape: 1536.0988 - mae: 0.0225 - val_loss: 0.0060 - val_mape: 27.8638 - val_mae: 0.0563
Epoch 100/100
1105/1105 [==============================] - 8s 8ms/step - loss: 8.8493e-04 - mape: 1489.6116 - 

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 874.1745657920837


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 5ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.1791585 , 0.15095751, 0.13883707, ..., 0.34902343, 0.31317148,
       0.28735515], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.31389866, 0.27683097, 0.25978835, ..., 0.34688831, 0.31152487,
       0.2780483 ])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,1284.8,1063.430054
1,1223.9,1017.097473
2,1195.9,997.184326
3,1190.7,996.312134
4,1211.9,1015.287720
...,...,...
4363,1452.6,1503.224243
4364,1399.7,1426.651123
4365,1339.0,1342.507935
4366,1280.9,1283.605347


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

6.940830472504683